### 데이터 수집

In [9]:
import requests
import time
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# --------------------------------
# 설정
# --------------------------------
API_KEY = os.getenv('ETHEREUM_API_KEY')  # Etherscan API 키 입력
CONTRACT = os.getenv('ETHEREUM_TOKEN_CONTRACT_ADDRESS')  # PicaArtMoney 토큰 컨트랙트
START_DATE = "2020-10-13"
END_DATE   = "2024-08-07"

In [14]:
# --------------------------
# 유틸: UTC 타임스탬프 변환
# --------------------------
def to_ts_utc(dstr: str) -> int:
    y, m, d = map(int, dstr.split("-"))
    return int(datetime(y, m, d, tzinfo=timezone.utc).timestamp())

start_ts = to_ts_utc(START_DATE)
end_ts   = to_ts_utc(END_DATE)

# --------------------------
# 블록 번호 by timestamp
# --------------------------
def get_block_number_by_timestamp(ts: int, closest="before") -> int:
    url = "https://api.etherscan.io/api"
    params = {
        "module": "block",
        "action": "getblocknobytime",
        "timestamp": ts,
        "closest": closest,
        "apikey": API_KEY
    }
    r = requests.get(url, params=params, timeout=30).json()
    return int(r["result"])

start_block = get_block_number_by_timestamp(start_ts, "after")
end_block   = get_block_number_by_timestamp(end_ts, "before")
print(f"[INFO] Block range: {start_block} ~ {end_block}")

# --------------------------
# 페이지 호출 (재시도 포함)
# --------------------------
def fetch_transfers_page(page, offset, start_block, end_block):
    url = "https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": CONTRACT,
        "startblock": start_block,
        "endblock": end_block,
        "page": page,
        "offset": offset,
        "sort": "asc",
        "apikey": API_KEY
    }
    # 간단 재시도
    for i in range(5):
        try:
            resp = requests.get(url, params=params, timeout=30).json()
            # status=="1"이면 결과, "0"이면 없음(또는 rate limit)
            if resp.get("status") == "1":
                return resp["result"]
            # "Max rate limit reached" 같은 메시지면 잠깐 대기 후 재시도
            if "Max rate limit reached" in resp.get("message",""):
                time.sleep(1.0 * (i+1))
                continue
            return []
        except Exception:
            time.sleep(1.0 * (i+1))
    return []

# --------------------------
# 범위 수집 (빈 청크도 로그)
# --------------------------
all_txs = []
step = 200_000        # 블록 청크 크기
offset = 10_000       # 페이지 크기
sleep_s = 0.25        # rate limit 보호
current = start_block

# (선택) 처리한 청크 목록 보관해 누락 시각화
processed_ranges = []

while current <= end_block:
    to_block = min(current + step - 1, end_block)
    processed_ranges.append((current, to_block))

    page = 1
    new_cnt = 0
    while True:
        txs = fetch_transfers_page(page, offset, current, to_block)
        if not txs:
            # 첫 페이지부터 빈 경우 → 이 청크엔 트랜잭션 없음을 명시
            if page == 1:
                print(f"[INFO] Blocks {current}–{to_block}, Page {page}, NO TX")
            break

        # 기간 필터
        for tx in txs:
            ts = int(tx.get("timeStamp", 0))
            if start_ts <= ts <= end_ts:
                all_txs.append(tx)
                new_cnt += 1

        print(f"[INFO] Blocks {current}–{to_block}, Page {page}, Added {new_cnt}, Total {len(all_txs)}")
        page += 1
        time.sleep(sleep_s)

        # 마지막 페이지(=반환건수 < offset) 판단 → 더 이상 이 청크에서 페이지 없음
        if len(txs) < offset:
            break

    current = to_block + 1  # 다음 청크로

# --------------------------
# DataFrame & 중복 제거
# --------------------------
df = pd.DataFrame(all_txs)

# 어떤 컬럼이 있는지 확인
print("[INFO] Columns:", df.columns.tolist())

# 존재하는 컬럼만으로 중복 제거
subset_cols = [c for c in ["hash", "logIndex", "transactionIndex"] if c in df.columns]
if subset_cols:
    df.drop_duplicates(subset=subset_cols, inplace=True)
else:
    df.drop_duplicates(subset=["hash"], inplace=True)

# (선택) amount 컬럼 추가
if {"value","tokenDecimal"} <= set(df.columns):
    df["amount"] = (df["value"].astype("int64") /
                    (10 ** df["tokenDecimal"].astype(int)))
else:
    df["amount"] = 0.0

out_path = "picaartmoney_transactions_full.csv"
df.to_csv(out_path, index=False)
print(f"[INFO] Finished. Total collected: {len(df)} transactions → {out_path}")

# --------------------------
# (선택) 누락 구간 시각 확인용
# --------------------------
# 빈 청크도 NO TX 로그가 찍히므로, 처리 범위가 연속적이라면 누락은 없습니다.
# 그래도 안심용으로 연속성 체크:
gaps = []
for (a1,b1),(a2,b2) in zip(processed_ranges, processed_ranges[1:]):
    if a2 != b1 + 1:
        gaps.append((b1+1, a2-1))
if gaps:
    print("[WARN] Detected block gaps in iteration (should not happen):", gaps)
else:
    print("[INFO] No block range gaps in iteration (every chunk covered).")

[INFO] Block range: 11043877 ~ 20472970
[INFO] Blocks 11043877–11243876, Page 1, Added 3828, Total 3828
[INFO] Blocks 11243877–11443876, Page 1, Added 238, Total 4066
[INFO] Blocks 11443877–11643876, Page 1, Added 321, Total 4387
[INFO] Blocks 11643877–11843876, Page 1, Added 2973, Total 7360
[INFO] Blocks 11843877–12043876, Page 1, Added 2121, Total 9481
[INFO] Blocks 12043877–12243876, Page 1, Added 5185, Total 14666
[INFO] Blocks 12243877–12443876, Page 1, Added 2273, Total 16939
[INFO] Blocks 12443877–12643876, Page 1, Added 1210, Total 18149
[INFO] Blocks 12643877–12843876, Page 1, Added 3398, Total 21547
[INFO] Blocks 12843877–13043876, Page 1, Added 1086, Total 22633
[INFO] Blocks 13043877–13243876, Page 1, Added 220, Total 22853
[INFO] Blocks 13243877–13443876, Page 1, Added 144, Total 22997
[INFO] Blocks 13443877–13643876, Page 1, Added 3338, Total 26335
[INFO] Blocks 13643877–13843876, Page 1, Added 38, Total 26373
[INFO] Blocks 13843877–14043876, Page 1, Added 10, Total 2638

In [16]:
# ============================================================
# 0) Imports
# ============================================================
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

from torch_geometric.data import Data
from torch_geometric.nn import GATConv, GAE
from torch_geometric.utils import to_undirected, negative_sampling
from torch_geometric.transforms import RandomLinkSplit

# ------------------------------------------------------------
# 파일 경로
CSV_PATH = "picaartmoney_transactions_full.csv"

# 상위 몇 개 이상노드 출력/저장
TOPK = 200

In [17]:
# ============================================================
# 1) CSV 로드 & 전처리
#    - 노드: address(지갑)
#    - 엣지: from -> to
#    - 금액: value / (10 ** tokenDecimal)
# ============================================================
df = pd.read_csv(CSV_PATH, dtype=str)

# 필요한 컬럼 존재 확인 후 숫자 변환
for col in ["from","to"]:
    if col not in df.columns:
        raise ValueError(f"CSV에 '{col}' 컬럼이 없습니다.")
df["from"] = df["from"].str.lower().fillna("")
df["to"]   = df["to"].str.lower().fillna("")

# 숫자 컬럼들
num_cols_maybe = ["value","tokenDecimal","gasPrice","timeStamp"]
for c in num_cols_maybe:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# amount 계산 (없으면 0)
if "value" in df.columns and "tokenDecimal" in df.columns:
    dec = df["tokenDecimal"].fillna(0).replace(0, 1).astype(int)
    df["amount"] = (df["value"].fillna(0) / (10.0 ** dec))
else:
    df["amount"] = 0.0

# gasPrice / timeStamp 없으면 0
if "gasPrice" not in df.columns:
    df["gasPrice"] = 0.0
if "timeStamp" not in df.columns:
    df["timeStamp"] = 0

In [18]:
# ============================================================
# 2) 엣지 집계 (from,to 단위 중복 전송 합치기)
# ============================================================
grp = df.groupby(["from","to"], dropna=False).agg(
    cnt=("from","size"),
    amt=("amount","sum")
).reset_index()

# 주소 목록 & 매핑
nodes = pd.Index(sorted(set(grp["from"]).union(set(grp["to"]))))
addr2id = {a:i for i,a in enumerate(nodes)}
num_nodes = len(nodes)

# 엣지 id 매핑
edges_df = grp.copy()
edges_df["src_id"] = edges_df["from"].map(addr2id)
edges_df["dst_id"] = edges_df["to"].map(addr2id)
edges_df.dropna(subset=["src_id","dst_id"], inplace=True)
edges_df["src_id"] = edges_df["src_id"].astype(int)
edges_df["dst_id"] = edges_df["dst_id"].astype(int)

print(f"[INFO] nodes: {num_nodes:,}, edges(unique from→to): {len(edges_df):,}")

[INFO] nodes: 7,958, edges(unique from→to): 14,128


In [19]:
# ============================================================
# 3) 노드 통계 피처 만들기 (행동 통계)
#    - out_cnt/in_cnt/out_amt/in_amt/avg_gas/활동기간 등 → 로그 스케일
# ============================================================
node_stats = {}

# out 통계
out_grp = df.groupby("from").agg(
    out_cnt=("from","size"),
    out_amt=("amount","sum"),
    avg_gas_out=("gasPrice","mean"),
    first_ts=("timeStamp","min"),
    last_ts=("timeStamp","max"),
).reset_index().rename(columns={"from":"addr"})

for _, r in out_grp.iterrows():
    s = node_stats.setdefault(r["addr"], {"out_cnt":0,"in_cnt":0,"out_amt":0.0,"in_amt":0.0,
                                          "avg_gas":0.0,"first_ts":1e18,"last_ts":0})
    s["out_cnt"] += int(r["out_cnt"]) if pd.notna(r["out_cnt"]) else 0
    s["out_amt"] += float(r["out_amt"]) if pd.notna(r["out_amt"]) else 0.0
    if pd.notna(r["avg_gas_out"]):
        s["avg_gas"] = (s["avg_gas"] + float(r["avg_gas_out"])) / 2.0 if s["avg_gas"] > 0 else float(r["avg_gas_out"])
    s["first_ts"] = min(s["first_ts"], int(r["first_ts"])) if pd.notna(r["first_ts"]) else s["first_ts"]
    s["last_ts"]  = max(s["last_ts"],  int(r["last_ts"]))  if pd.notna(r["last_ts"])  else s["last_ts"]

# in 통계
in_grp = df.groupby("to").agg(
    in_cnt=("to","size"),
    in_amt=("amount","sum"),
    avg_gas_in=("gasPrice","mean"),
    first_ts=("timeStamp","min"),
    last_ts=("timeStamp","max"),
).reset_index().rename(columns={"to":"addr"})

for _, r in in_grp.iterrows():
    s = node_stats.setdefault(r["addr"], {"out_cnt":0,"in_cnt":0,"out_amt":0.0,"in_amt":0.0,
                                          "avg_gas":0.0,"first_ts":1e18,"last_ts":0})
    s["in_cnt"] += int(r["in_cnt"]) if pd.notna(r["in_cnt"]) else 0
    s["in_amt"] += float(r["in_amt"]) if pd.notna(r["in_amt"]) else 0.0
    if pd.notna(r["avg_gas_in"]):
        s["avg_gas"] = (s["avg_gas"] + float(r["avg_gas_in"])) / 2.0 if s["avg_gas"] > 0 else float(r["avg_gas_in"])
    s["first_ts"] = min(s["first_ts"], int(r["first_ts"])) if pd.notna(r["first_ts"]) else s["first_ts"]
    s["last_ts"]  = max(s["last_ts"],  int(r["last_ts"]))  if pd.notna(r["last_ts"])  else s["last_ts"]

# DataFrame으로
ns = pd.DataFrame.from_dict(node_stats, orient="index")
ns.index.name = "addr"
ns.reset_index(inplace=True)

# 누락 주소(엣지엔 있는데 통계가 비어있는 경우) 채워넣기
missing = list(set(nodes) - set(ns["addr"]))
if missing:
    ns2 = pd.DataFrame({
        "addr": missing,
        "out_cnt": 0, "in_cnt": 0, "out_amt": 0.0, "in_amt": 0.0,
        "avg_gas": 0.0, "first_ts": 0, "last_ts": 0
    })
    ns = pd.concat([ns, ns2], ignore_index=True)

ns["node_id"] = ns["addr"].map(addr2id)
ns = ns.dropna(subset=["node_id"]).sort_values("node_id")
ns["node_id"] = ns["node_id"].astype(int)

# 결측 0, degree, 로그 스케일
for c in ["out_cnt","in_cnt","out_amt","in_amt","avg_gas"]:
    ns[c] = ns[c].fillna(0.0)
ns["deg"] = ns["out_cnt"] + ns["in_cnt"]

for c in ["out_cnt","in_cnt","out_amt","in_amt","deg","avg_gas"]:
    ns[f"log_{c}"] = np.log1p(ns[c])

# 활동 기간
ns["life_span"] = (ns["last_ts"] - ns["first_ts"]).clip(lower=0)
ns["log_life_span"] = np.log1p(ns["life_span"])

feat_cols = ["log_out_cnt","log_in_cnt","log_out_amt","log_in_amt","log_deg","log_avg_gas","log_life_span"]
x_mat = np.zeros((num_nodes, len(feat_cols)), dtype=np.float32)
x_mat[ns["node_id"].values, :] = ns[feat_cols].values.astype(np.float32)

In [20]:
# ============================================================
# 4) PyG Data 구성
# ============================================================
edge_index = torch.tensor(
    np.vstack([edges_df["src_id"].values, edges_df["dst_id"].values]),
    dtype=torch.long
)
# 무방향으로 맞춰 간단화 (GAE의 inner product 디코더와 잘 맞음)
edge_index = to_undirected(edge_index, num_nodes=num_nodes)

x = torch.tensor(x_mat, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[7958, 7], edge_index=[2, 27944])


In [21]:
# ============================================================
# 5) 데이터 분리 (메모리-안전) : RandomLinkSplit
#    - train/val/test로 링크를 분할, 음성샘플은 내부에서 처리
# ============================================================
splitter = RandomLinkSplit(
    num_val=0.05, num_test=0.10,
    is_undirected=True,
    add_negative_train_samples=False  # 학습시엔 recon_loss가 자체적으로 negative 샘플링 처리
)
train_data, val_data, test_data = splitter(data)

In [22]:
# ============================================================
# 6) GAT Encoder + GAE 정의
# ============================================================
class GATEncoder(nn.Module):
    def __init__(self, in_dim, hidden, out_dim, heads=4, dropout=0.2):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden*heads, out_dim, heads=1, dropout=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)  # GAT는 보통 ELU
        x = self.conv2(x, edge_index)  # 최종 임베딩 z (out_dim)
        return x

embed_dim = 16
encoder = GATEncoder(in_dim=x.size(1), hidden=32, out_dim=embed_dim, heads=4, dropout=0.2)
gae = GAE(encoder)

optimizer = torch.optim.Adam(gae.parameters(), lr=0.01, weight_decay=5e-4)

In [27]:
# ============================================================
# 7) 학습 루프 (링크 재구성)
# ============================================================
def train_epoch():
    gae.train()
    optimizer.zero_grad()
    z = gae.encode(train_data.x, train_data.edge_index)
    loss = gae.recon_loss(z, train_data.edge_index)  # 내부적으로 negative 포함 BCE
    loss.backward()
    optimizer.step()
    return float(loss.item())

EPOCHS = 100
for ep in range(1, EPOCHS+1):
    loss = train_epoch()
    if ep % 10 == 0:
        print(f"[Epoch {ep:3d}] recon loss: {loss:.4f}")

# ------------------------------------------------------------
# (안전) 링크 예측 평가
# ------------------------------------------------------------
from sklearn.metrics import roc_auc_score, average_precision_score

@torch.no_grad()
def evaluate(z, edge_label_index, edge_label):
    # 내적 기반 복원 확률
    logits = (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=1)
    prob = torch.sigmoid(logits).cpu().numpy()
    labels = edge_label.cpu().numpy()
    auc = roc_auc_score(labels, prob)
    ap = average_precision_score(labels, prob)
    return auc, ap

# 사용
z = gae.encode(val_data.x, train_data.edge_index)
auc, ap = evaluate(z, val_data.edge_label_index, val_data.edge_label)
print(f"[Val] AUC={auc:.4f}, AP={ap:.4f}")

[Epoch  10] recon loss: 22.4195
[Epoch  20] recon loss: 22.8467
[Epoch  30] recon loss: 22.6979
[Epoch  40] recon loss: 23.0294
[Epoch  50] recon loss: 22.4661
[Epoch  60] recon loss: 22.6689
[Epoch  70] recon loss: 22.5254
[Epoch  80] recon loss: 22.4237
[Epoch  90] recon loss: 22.2390
[Epoch 100] recon loss: 22.3835
[Val] AUC=0.6011, AP=0.5555


In [28]:
# ============================================================
# 8) 이상 점수 계산 (노드별 재구성 오차)
#    - 전체 NxN은 메모리 이슈 → 각 노드별로
#      * 양성: 그 노드의 실제 이웃들
#      * 음성: 무작위 K개(연결 안된 노드) 샘플
#    - BCE 평균을 노드 이상 점수로 사용
# ============================================================
@torch.no_grad()
def anomaly_scores_local(neg_k=100):
    gae.eval()
    # train 그래프 기준으로 임베딩
    z = gae.encode(train_data.x, train_data.edge_index)  # [N, d]
    N = z.size(0)

    # 인접리스트 만들기 (train 그래프 기준)
    ei = train_data.edge_index.cpu().numpy()
    adj = [[] for _ in range(N)]
    for s, t in zip(ei[0], ei[1]):
        adj[s].append(t)
        adj[t].append(s)  # undirected

    # 빠른 내적 계산 위해 z를 CPU 텐서로
    zc = z.cpu()
    scores = np.zeros(N, dtype=np.float64)

    # 음성 샘플링을 위해 전체 노드 인덱스 집합
    all_ids = np.arange(N)

    for i in range(N):
        pos_nei = np.array(adj[i], dtype=np.int64)
        # 양성(이웃)이 하나도 없으면, 음성만으로는 점수가 왜곡될 수 있어 0 처리 or skip
        if pos_nei.size == 0:
            scores[i] = 0.0
            continue

        # ----- 양성 로짓/확률 -----
        # p_ij = sigmoid(z_i^T z_j)
        zi = zc[i].unsqueeze(0)                         # [1, d]
        zj_pos = zc[pos_nei]                            # [deg(i), d]
        logit_pos = (zi @ zj_pos.T).squeeze(0)          # [deg(i)]
        prob_pos = torch.sigmoid(logit_pos).clamp(1e-7, 1-1e-7)
        bce_pos = -(torch.log(prob_pos)).mean().item()  # A=1 → -log(p)

        # ----- 음성 샘플링 -----
        # 후보: i 자신과 이웃을 제외한 나머지
        mask = np.ones(N, dtype=bool)
        mask[i] = False
        mask[pos_nei] = False
        neg_pool = all_ids[mask]
        if neg_pool.size == 0:
            bce_neg = 0.0
        else:
            take = min(neg_k, neg_pool.size)
            neg_ids = np.random.choice(neg_pool, size=take, replace=False)
            zj_neg = zc[neg_ids]                         # [K, d]
            logit_neg = (zi @ zj_neg.T).squeeze(0)       # [K]
            prob_neg = torch.sigmoid(logit_neg).clamp(1e-7, 1-1e-7)
            bce_neg = -(torch.log(1 - prob_neg)).mean().item()  # A=0 → -log(1-p)

        # 노드 i의 이상 점수 = 양성/음성 BCE 평균
        scores[i] = (bce_pos + bce_neg) / 2.0

    return scores

scores = anomaly_scores_local(neg_k=100)

# 상위 TOPK 저장/출력
id2addr = {i:a for a,i in addr2id.items()}
top_idx = np.argsort(-scores)[:min(TOPK, len(scores))]
top_df = pd.DataFrame({
    "node_id": top_idx,
    "address": [id2addr[i] for i in top_idx],
    "anomaly_score": scores[top_idx]
})
top_df.to_csv("pica_anomalous_wallets_top.csv", index=False)
print("[INFO] Saved:", "pica_anomalous_wallets_top.csv")
print(top_df.head(10))

[INFO] Saved: pica_anomalous_wallets_top.csv
   node_id                                     address  anomaly_score
0     3978  0x7e628e98b2e2b70d0a69c1de43ae15a43ec38202       7.971192
1     5247  0xa6baffd3d39f5d114c3ec54b74924cf46acb1246       7.971192
2     5246  0xa6a205f6de916c4f60336b04aa95799ae1e0fdcf       7.971192
3     5245  0xa68e8fe312560ccb0e35e1c3bab358d4215d6af6       7.971192
4     5244  0xa683b90182886fe5051c37a128852a0ad8d2db93       7.971192
5     5243  0xa67af3850f244390fd6a725e49b39ba36c007b76       7.971192
6     5242  0xa674737ef8cac3fe2b38446a902ad26516281133       7.971192
7     5241  0xa6725a72e7ccc332b844ed1eef1e08867c14eec6       7.971192
8     5240  0xa66274261b7bd155a8ce7ea9865798a4b652a2f7       7.971192
9     5239  0xa647b5d11e8bfc9d7b75160372ce4490007ad8b2       7.971192


In [29]:
df

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,methodId,functionName,confirmations,amount
0,11085582,1603098539,0x328008e17407b6ba014295bfe5069ba4f60f1296aea0...,0,0x7185066660404b22f7f1cac0862c8a7f5c1ef99e4a1f...,0x0000000000000000000000000000000000000000,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xd28493e737fbcc957f3716143ed6e40f40357b51,1000000000,PicaArtMoney,...,251,1603895,36000000000,1603895,11088223,deprecated,0x60806040,"atInversebrah(int248 a, uint48[] b, uint32 c, ...",12287913,100000000.0
1,11091306,1603174037,0xa8352e8094fb444e9bfa9a4a6d8011502a0f4655ad38...,1,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xfa9b57cbe5b7bd63b436dcf205c15222b510ff27,150000000,PicaArtMoney,...,185,53110,26000000000,53110,12360057,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",12282189,15000000.0
2,11091306,1603174037,0x36195c14399493bdf43387ded77d87d3b5e98c94a138...,2,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0x32f042b0b01f10247493a950456f4c4304d46ba5,150000000,PicaArtMoney,...,186,53110,26000000000,53110,12413167,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",12282189,15000000.0
3,11091306,1603174037,0xbf52a2c6de897287b5c6740c891a71a5122c69566bdb...,3,0x6902d2e0773fd0bad116b753b98e37d61d7a4b532d71...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xd4b394c60bb55f80df30dac87b6f92be34739332,300000000,PicaArtMoney,...,187,53098,26000000000,53098,12466265,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",12282189,30000000.0
4,11091313,1603174087,0x246a4998919d4c369ff0735ea372729a9e06199055e1...,4,0xe18ff0b2e114d21a3f5cf8c2976342cea2fc558254ee...,0xd28493e737fbcc957f3716143ed6e40f40357b51,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xc32b1345acae345c595d3bbcf62e14e5f3020456,200000000,PicaArtMoney,...,100,53098,27000000000,53098,5923677,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",12282182,20000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25598,19890680,1715960855,0xc38256fa631db46712fa75e30075aa8bc42810728a78...,84710,0x0cf33dc1e537de47fed77085765f028d2a661ccb8391...,0x429cf888dae41d589d57f6dc685707bec755fe63,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,130860,PicaArtMoney,...,3,207205,71652762134,145044,771708,deprecated,0x0300000e,NaN,3482820,13086.0
25599,20250756,1720308515,0xe2467f46394bcfbeb290042c71c7742c72b977f53eed...,10,0x879a20d069dc2a710f7ce843d6fd10fb98dd4ce36a84...,0x370b453caa04583317a4543947c5568b48b31567,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0x9763d6413141e23a32d1459fe7cfde6a4cddcb36,756,PicaArtMoney,...,80,54382,1651237587,49386,12073787,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",3122745,75.6
25600,20256944,1720383179,0x91f8c2a6da8883c570d8fcc31f066403d1bf7945eb36...,4003,0x8be3934656e28e880f1345ecde0d0ac95cf08bc878a6...,0x9763d6413141e23a32d1459fe7cfde6a4cddcb36,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0x1d5a1eaf90218e91f2bb32e42b0b02ff39827d16,756,PicaArtMoney,...,153,308483,1288734825,172048,16414639,deprecated,0x3593564c,"execute(bytes commands,bytes[] inputs,uint256 ...",3116557,75.6
25601,20307085,1720988351,0xe26c0b65556d8657575f4e7575c498376bc51f8b71e2...,13,0xb22e28994698e6e09f318a0e98483d24b0ba5a55301c...,0xb5f756611eddfbd63f4e8d28f2a62a401431c35a,0xa7e0719a65128b2f6cdbc86096753ff7d5962106,0xa533574c3199db25bb91184ecc37e254fa08bee3,10,PicaArtMoney,...,82,54174,3483428097,54174,8084579,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",3066416,1.0
